In [ ]:
import os 
import re 
import json 
import string 
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
from keras import layers 
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer , TFBertModel , BertConfig 

max_len = 384 
configuration = BertConfig() # default parameters and configuration for Bert 


Set-up Bert tokenizer 

In [ ]:
# Save the slow pretrained tokenizer 
slow_tokenizer = BertTokenizer.from_pretrained('bert-base-unscased')
save_path = 'bert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file 
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)


Load the data

In [ ]:
train_data_url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json'
train_path = keras.utils.get_file('train.json', train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file('eval.json', eval_data_url)

Preprocessor the data

In [ ]:
class SquadExample:
    def __init__(self, question ,context, start_char_idx, answer_text, all_answer):
        self.question = question 
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answer = all_answer
        self.skip = False 
    def preprocess(self):
        context = self.context 
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question 
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())
    
    # Sẽ cần thực hiện các chức năng sau 
    # Tìm kiếm các chỉ số ký tự bắt đầu và kết thuc câu 
    # Và tạo ra các đầu vào cho mô hình 

        # 1 : Xây dựng phương thức tiềm kiếm vị trí ký tự cuối cùng trong câu trả lời
        # bằng cách lấy chỉ số ký tự đầu của câu trả lời + độ dài của toàn bộ xâu trả lời 
        # vd start = 17 và độ dài của 1 câu trả lời đó  = 7 
        #  24 - 1 
        end_char_idx = start_char_idx + len(answer)
        # kiểm tra xem chỉ số ký tự kết thúc có vượt quá độ dài ngữ cảnh 
        # Nếu có thì bỏ qua và thoát khỏi vòng lặp 
        if end_char_idx >= len(context):
            self.skip = True 
            return 
        
        # 2 : Đánh giấu các chỉ mục ký tự trong ngữ cảnh thuộc về câu trả lời 
        # và xác định các ký tự nào trong ngữ cảnh là thuộc cho câu tl 
        # và đánh dấu chúng = cách gán =  1 
        # Đầu tiên nhân tất cả giá trị ngữ cảnh của câu = 0 
        # Tức là sé sinh ra  1 vector có độ dài bằng với câu đấy vơis values = 0
        is_char_in_ans = [0] * len(context)
        # lặp qua câu thông qua ký tự đầu tiên và kết thúc ở ký tự tự cuối 
        # và gán cho các ký tự thuộc ctl = 1
        for idx in range(start_char_idx , end_char_idx):
            is_char_in_ans[idx] = 1 

        # 3 : Thực hiện token hóa ngữ cảnh 
        tokenized_context = tokenizer.encode(context)

        # 4 :tạo ra một danh sách các chỉ số của token mà thuộc về câu trả lời
        ans_token_idx = [] # nhận chỉ số vị trí của câu trả lời là vị trí token theo từ 
        # vd Việt Nam có vị trí 7 , 8 thì nó sẽ là [7,8]
        # Duyệt qua các token trong câu và lấy ra vị trí đâù và kết thúc cho mỗi token
        # từ offsets sẽ trả về vị trí bắt đầu và kết thúc cho mỗi token vd Viet (17,20) nam (21 ,23)
        # giúp cho việc đếm số thứ tự các phần tử trong danh sách vd (7,20) idx = 7 sẽ add 7 ,
        # (21 ,23)  = 8 add 8 => =[7 ,8]
        for idx , (start, end) in enumerate(tokenized_context.offsets):
            # nếu có chứa ít nhất một ký tự thuộc câu trả lời thì ta thêm chỉ số của token 
            if sum(is_char_in_ans[start:end] >0):
                ans_token_idx.append(idx) # IDX là chỉ số cho vị trí token
        
        # Kiểm tra xem danh sách có rỗng hay không 
        # Rỗng nghĩa là không token nào được tạo da từ ký tự của câu trả lời 
        if len(ans_token_idx) == 0:
            self.skip = True 
            return 
        
        # 5: Lấy ra vị trí bắt đầu và kết thúc cho toàn bộ câu trả lời 
        # Từ chỉ số đầu và cuối được lưu trữ từ vị trí token cho câu trả lời 
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # 6 : Tính toán token hóa câu hỏi 
        tokenized_question = tokenizer.encode(question)

        # 7: xây dựng lớp đầu vào cho mô hình bert lưu ý loại bỏ vị trí 0 là cls của question 
        # inputs =  tokenized_context.ids  + tokenized_question.ids[1:]
        # Sau đó thực hiện biến đổi lấp đầy 2 đoạn bơỉ 0 và 1
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        # 8 :khởi tạo mặt attention shape = inputs shape fully = 1
        attention_mask = [1] * len(input_ids)

        # 9 : Dệm và tạo mặt nạ chú ý 
        # bỏ qua nếu cần cắt bớt 
        # Tính toán độ dài phần đệm = độ dài tối đa cho phép (maxlen) - độ dài câu (input_ids)
        padding_length =  max_len - len(input_ids)
        # Nếu như padding_length > 0 thì ta cần thực hiện đệm và mã hóa cho đệm =0
        if padding_length > 0: # pad
            # thêm đệm vào input_ids và tham số của đệm = 0
            input_ids = input_ids + ( [0] * padding_length)
            # tương tự với mặt nạ chú ý và token_type 
            attention_mask = attention_mask + ( [0] * padding_length)
            token_type_ids = token_type_ids + ( [0] * padding_length)
        # Nếu độ dài câu = max len thì bỏ qua ko 
        elif padding_length < 0:
            self.skip = True 
            return 
        
        # 10 : trả về loạt các danh sách 
        # đầu vào 
        self.input_ids = input_ids # shape = interger parameter + padding 0
        self.token_type_ids = token_type_ids # shape = 0 , 1 + padding 0
        self.attention_mask= attention_mask # hape =  1 + padding 0
        self.start_token_idx = start_token_idx # là vị trí bắt đầu của câu tl theo vị trí các token
        self.end_token_idx = end_token_idx # là vị trí kết thúc của câu tl theo vị trí các token trong câu 
        # Cuối cùng là trả về vị trí bát đầu và kết thúc của token được giới hạn bởi 
        # số ký tự có trong token vd viet = (17 , số ký tự  = 4) - 1 = (17,20)
        self.context_token_to_char = tokenized_context.offsets 


# đọc dữ liệu huấn luyện và đọc dữ liệu thẩm đinh 
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


# Khởi tạo đội hình ví dụ dựa vào data train 
# gồm question , answer , all_answer , start_char_idx 
# sau đó xử lý các vector 

def create_squad_examples(raw_data):
    squad_examples = []
    # Duyệt qua các phần tử trong raw_data['data] mỗi phần tử sẽ tương tự với 1 bài 
    # viết trong wiki và chứa nhiều đoạn văn bản 
    for item in raw_data['data']:
        # Duyệt qua các đoạn đoạn văn bản trong item["paragraphs"]
        for para in item['paragraphs']:
            # lấy ra các đoạn văn bản mỗi đoạn chứa 1 chuỗi context 
            context = para['context']
            # Duyêt qua một danh sách qa gồm các câu hỏi và câu tl 
            for qa in para['qas']:
                # Lấy ra Chuỗi câu hỏi được đặt ra dựa trên context.
                question = qa['question']
                # Chuỗi câu trả lời đúng cho câu hỏi, được lấy ra từ context.
                answer_text = qa['answers'][0]['text'] 
                # Lấy ra danh sách các chuỗi câu trả lời đúng cho câu hỏi 
                # có thể có nhiều cách trả lời khác nhau cho một câu hỏi 
                all_answers = [ om['text']  for om in qa['answers']]
                # lấy ra vị chí của ký tự bắt đầu cho câu trả lời 
                start_char_idx = qa['answers'][0]['answer_start']

                # Thực hiện tiền xử lý dữ liệu thô 
                squad_eg = SquadExample(
                    question , context, start_char_idx , answer_text , all_answers
                )
                squad_eg.preprocess()
                squad_examples.append( squad_eg)

    return squad_examples

# Xây dựng bộ phần xử lý đầu vào từ danh sách squad_examples 

def create_inputs_targets(squad_examples):
    dataset_dict = {
        'inputs_ids' : []
        ,'token_type_ids':[],
        "attention_mask": [],
        "start_token_idx" :[],
        "end_token_idx" : [],
    }
    # Duyệt qua đối tượng squad_example để lấy ra các item chứa trong nó 
    for item in squad_examples:
        # nếu các item không được bỏ qua thì duyệt qua các key của từ điển tạo sẵn 
        if item.skip == False :
            for key in dataset_dict:
                # Thêm các item của squad _example vào với từ điển chứa các từ khóa 
                # tương ứng  , mỗi từ điển sẽ chứa 1 danh sách tương ứng với nó 
                # Sử dụng hàm getattr để lấy giá trị của thuộc tính đối tượng 
                # áp dụng cho từ key một 
                dataset_dict[key].append(getattr(item, key))
    
    # duyệt qua các key trong từ điển và biến đổi nó thành mảng aray 
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
    
    # tạo x và y để phục vụ cho huấn luyện mô hình 
    x = [
        dataset_dict["inputs_ids"],
        dataset_dict['token_type_ids'],
        dataset_dict['attention_mask'],
    ]
        
    y = [
        dataset_dict['start_token_idx'],
        dataset_dict [' end_token_idx'],
    ]
    return x , y


# Tiến hành sử lý hóa dữ liệu từ train và eval cho mô hình 
train_squad_examples = create_squad_examples(raw_train_data)
x_train , y_train = create_inputs_targets(train_squad_examples)

print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval , y_eval=create_inputs_targets(eval_squad_examples)

print(f"{len(eval_squad_examples)} evaluation points created.") 


Create the Question-Answering Model using BERT and Functional API

In [ ]:
def create_model():
    # Bert encoder
    encoder = TFBertModel.from_pretrained('bert-base-uncased')
    # QA model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), stype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logist = layers.Dense(units=1 , name='start_logit', use_bias=False)(embedding)
    start_logist = layers.Flatten()(start_logist)

    end_logits = layers.Dense(units=1 , name='end_logit', use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logist)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask], outputs=[start_probs, end_probs]
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = tf.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model


In [ ]:
use_tpu = True 
if use_tpu:
    # create distribution stratery 
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

Create evaluation Callback

In [ ]:
def normalize_text(text):
    text = text.lower()
    # Remove punctuations xóa dấu câu
    exclude = set(string.punctuation)
    # sau đó nối lại các ký tự trong văn bản và không thuộc tập exclude 
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles sử dụng biên dịch biểu thức 
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    # sử dụng re.sub để thay thế các ký tự =  " "  và trả về 1 text 
    text = re.sub(regex, ' ', text)

    # Xoa khoẳng trắng thừa 
    text =  " ".join(text.split())
    return text 

class ExactMatch(keras.callbacks.Callback):
    """
    Mỗi đối tượng `SquadExample` chứa độ lệch cấp độ ký tự cho mỗi mã thông báo
    trong đoạn đầu vào của nó. Chúng tôi sử dụng chúng để lấy lại khoảng văn bản tương ứng
    đến các mã thông báo giữa mã thông báo bắt đầu và kết thúc được dự đoán .
    Tất cả các câu trả lời thực tế cũng có trong mỗi đối tượng `SquadExample`.
    tính toán tỷ lệ phần trăm của các điểm dữ liệu mà khoảng văn bản thu được
    từ các dự đoán mô hình phù hợp với một trong những câu trả lời đúng sự thật.
        """
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self , epoch, logs=None):
        pred_start , pred_end = self.model.predict(self.x_eval)
        # Khởi tạo cuont để đếm số lượng câu trả lời chính xác 
        count = 0
        # Lọc qua những vi dụ trong tập dư liệu kiểm tra eval_squad_examples mà không
        # bị bỏ qua 
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        # Duyệt qua những cặp vị trí bắt đầu và kết thúc dự đoán cùng với những chỉ số tương ứng 
        for idx , (start , end) in enumerate(zip(pred_start , pred_end)):
            
            # Lấy ra ví dụ tương ứng trong tập dữ liệu kiểm tra đã lọc
            squad_eg = eval_examples_no_skip[idx]
            # lấy ra danh sách cac vị trí ký tự trong đoạn văn bản tương ứng với từng token
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            # Nếu vị trí bắt đầu lớn hơn hoặc bằng độ dài của danh sách offsets,
            # phương thức này sẽ bỏ qua ví dụ này và tiếp tục vòng lặp
            # tức là dự đoán không thuộc offsets
            if start >=  len(offsets):
                continue
            # Nếu không, lấy ra vị trí ký tự bắt đầu của token bắt đầu 
            pred_char_start = offsets[start][0]
            # Nếu vị trí kết thúc nhỏ hơn độ dài của danh sách offsets, phương thức này sẽ lấy ra 
            # vị trí ký tự bắt đầu và kết thúc của câu trả lời trong văn bản ngữ cảnh 
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                # thực hiện khoanh vung cho câu tl
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else : 
            # Ngược lại ta lấy từ vị trí ký tự dự đoán đầu tiên đến hết câu . 
                pred_ans = squad_eg.context[pred_char_start:]
            
            # Chuẩn hóa câu trả lời dự đoán
            normalized_pred_ans = normalize_text(pred_ans)
            # tạo danh sách chuẩn hóa tất cả các nhãn thực 
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            
            # nếu như các nhãn dự đoán mà thuộc nhãn thực thì tăng count lên 1 
            if normalized_pred_ans in normalized_true_ans:
                count += 1 
        # Sau khi duyệt hết các ví dụ kiểm tra, phương thức này sẽ tính toán độ chính xác của mô hình
        # bằng cách chia số lượng câu trả lời chính xác cho số lượng ví dụ kiểm tra
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")



In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
model.fit(
    x_train,
    y_train,
    epochs=1,  # For demonstration, 3 epochs are recommended
    verbose=2,
    batch_size=64,
    callbacks=[exact_match_callback],
)